# Heart Attack Risk Level Assessment Machine Learning Model .

# Dependencies Installation

In [ ]:
!pip install scikit-learn tensorflow numpy pandas

# 📢 **IMPORTANT**

**Download the Dataset from the Github Repository else you will encounter error and program will not execute .**

DataSet Name: Heart_Attack_Risk_Levels_Dataset.csv

Dataset Link: https://github.com/XORO1337/DNN_Attack


 # Be Patient As It might Take Time to Train the Model .

 If You Wanna reduce the training time, you can tweak the `EPOCHS = 600` value
 ( Reduce Epoch value ▶ Faster Program )

# ...................... MAIN CODE .........................


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
tf.config.run_functions_eagerly(True)
import os
import pickle

# Model and data preparation constants
MODEL_FILE = "heart_attack_model.h5"
SCALER_FILE = "scaler.pkl"
ENCODER_FILE = "label_encoder.pkl"
TRAINING_DATA_FILE = "training_data.pkl"

def load_or_train_model():
    df = pd.read_csv("Heart_Attack_Risk_Levels_Dataset.csv")

    feature_columns = ['Age', 'Gender', 'Heart rate', 'Systolic blood pressure',
                       'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin']
    X = df[feature_columns]
    y = df['Risk_Level']

    recommendations = df[['Risk_Level', 'Recommendation']].drop_duplicates().set_index('Risk_Level')['Recommendation'].to_dict()

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    if os.path.exists(MODEL_FILE) and os.path.exists(SCALER_FILE) and os.path.exists(ENCODER_FILE) and os.path.exists(TRAINING_DATA_FILE):
        print("📁 Loading existing model and data...")
        model = tf.keras.models.load_model(MODEL_FILE)

        with open(SCALER_FILE, 'rb') as f:
            scaler = pickle.load(f)

        with open(ENCODER_FILE, 'rb') as f:
            label_encoder = pickle.load(f)

        with open(TRAINING_DATA_FILE, 'rb') as f:
            training_data = pickle.load(f)
            X_train = training_data['X_train']
            y_train = training_data['y_train']
            X_test = X_train
            y_test = y_train
    else:
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(16, activation='relu', input_shape=(X.shape[1],)),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dense(3, activation='softmax')
        ])

        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        EPOCHS = 600 # Tweak them according to your needs ........
        print("🧠 Training the model...\n")
        history = model.fit(X_train, y_train, validation_split=0.1, epochs=EPOCHS, verbose=0)

        for i in range(0, EPOCHS, 50):
            print(f"Epoch {i+1} - Loss: {history.history['loss'][i]:.4f}, Accuracy: {history.history['accuracy'][i]*100:.2f}%, Val Acc: {history.history['val_accuracy'][i]*100:.2f}%")

        # Plot training and validation accuracy
        import matplotlib.pyplot as plt
        plt.figure(figsize=(8, 5))
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Training vs Validation Accuracy')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"\n✅ Final Test Accuracy: {accuracy * 100:.2f}%")

        model.save(MODEL_FILE)
        with open(SCALER_FILE, 'wb') as f:
            pickle.dump(scaler, f)
        with open(ENCODER_FILE, 'wb') as f:
            pickle.dump(label_encoder, f)
        with open(TRAINING_DATA_FILE, 'wb') as f:
            pickle.dump({'X_train': X_train, 'y_train': y_train}, f)

    return model, scaler, label_encoder, recommendations, feature_columns, X_test, y_test



# Function to collect user input
def get_user_input(feature_columns):
    print("\nPlease answer the following questions:")
    inputs = []
    inputs.append(float(input("Age (in years): ")))
    inputs.append(int(input("Gender (0 = Female, 1 = Male): ")))
    inputs.append(float(input("Heart Rate (bpm): ")))
    inputs.append(float(input("Systolic Blood Pressure (mmHg): ")))
    inputs.append(float(input("Diastolic Blood Pressure (mmHg): ")))
    inputs.append(float(input("Blood Sugar (mg/dL): ")))
    inputs.append(float(input("CK-MB (ng/mL): ")))
    inputs.append(float(input("Troponin (ng/mL): ")))

    # Create a dictionary with feature names
    input_dict = {feature: value for feature, value in zip(feature_columns, inputs)}
    return np.array(inputs).reshape(1, -1), input_dict

# Function to predict and learn from new input
def predict_risk_and_learn(model, scaler, label_encoder, recommendations, feature_columns):
    user_input, input_dict = get_user_input(feature_columns)
    user_input_scaled = scaler.transform(user_input)

    # Predict using the model
    prediction = model.predict(user_input_scaled, verbose=0)
    risk_index = np.argmax(prediction)
    risk_label = label_encoder.inverse_transform([risk_index])[0]
    recommendation = recommendations[risk_label]

    print(f"\n💡 Predicted Heart Attack Risk Level: **{risk_label}**")
    print(f"📋 Recommendation: {recommendation}")

    # Show prediction confidence
    confidence = prediction[0][risk_index] * 100
    print(f"🔍 Confidence: {confidence:.2f}%")

    # Print the user's input values for reference
    print("\nYour input values:")
    for feature, value in input_dict.items():
        print(f"- {feature}: {value}")

    # Ask the user if the prediction was correct
    feedback = input("\nWas this prediction correct? (yes/no): ").strip().lower()
    if feedback == "yes":
        print("✅ Great! No update needed.")
        return
    elif feedback == "no":
        # Get the actual risk level from the user
        actual = input("Please enter the correct risk level (Low / Moderate / High): ").capitalize()

        if actual in label_encoder.classes_:
            actual_encoded = label_encoder.transform([actual])[0]

            # Load existing training data
            with open(TRAINING_DATA_FILE, 'rb') as f:
                training_data = pickle.load(f)
                X_train = training_data['X_train']
                y_train = training_data['y_train']

            # Add new data point to training set
            X_train_updated = np.vstack([X_train, user_input_scaled])
            y_train_updated = np.append(y_train, actual_encoded)

            # Update the saved training data
            training_data = {'X_train': X_train_updated, 'y_train': y_train_updated}
            with open(TRAINING_DATA_FILE, 'wb') as f:
                pickle.dump(training_data, f)

            # Retrain the model briefly to include new input
            print("⏳ Retraining model with your corrected input...")
            model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy')
            model.fit(user_input_scaled, np.array([actual_encoded]), epochs=200, verbose=0)

            # Save the updated model
            model.save(MODEL_FILE)
            model = tf.keras.models.load_model(MODEL_FILE)
            print("✅ Model updated and saved with your correction.")

            # Predict again to show the effect of learning
            updated_prediction = model.predict(user_input_scaled, verbose=0)
            updated_risk_index = np.argmax(updated_prediction)
            updated_risk_label = label_encoder.inverse_transform([updated_risk_index])[0]
            updated_confidence = updated_prediction[0][updated_risk_index] * 100

            print(f"\n🔄 After retraining, your risk level would now be predicted as: **{updated_risk_label}**")
            print(f"🔍 Updated confidence: {updated_confidence:.2f}%")
        else:
            print("⚠️ Invalid input. Skipping model update.")
    else:
        print("⚠️ Invalid input. Skipping model update.")

# Main program
def main():
    print("💓 Heart Attack Risk Prediction System 💓")
    model, scaler, label_encoder, recommendations, feature_columns, X_test, y_test = load_or_train_model()

    # 📊 CONFUSION MATRIX VISUALIZATION
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    import matplotlib.pyplot as plt
    import numpy as np

    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred_labels = np.argmax(y_pred_probs, axis=1)

    cm = confusion_matrix(y_test, y_pred_labels)
    labels = label_encoder.classes_

    plt.figure(figsize=(6, 5))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap='Blues', values_format='d')
    plt.title("Confusion Matrix - Heart Attack Risk Prediction")
    plt.grid(False)
    plt.tight_layout()
    plt.show()

    # User interaction loop
    while True:
        ask = input("\nWould you like to assess your heart attack risk? (yes/no): ").strip().lower()
        if ask == "yes":
            predict_risk_and_learn(model, scaler, label_encoder, recommendations, feature_columns)
        else:
            print("Goodbye! Stay healthy. 💖")
            break


if __name__ == "__main__":
    main()



# ................... WEB APP BACKEND ..........................

In [ ]:
!pip install streamlit pyngrok flask

In [ ]:
!ngrok config add-authtoken 27ABnJRKuCiZI9XiyqjLVu0lhjt_6QB8f9m7yJGCad38xiXEr

In [ ]:
import os
import numpy as np
import pickle
import tensorflow as tf
from flask import Flask, render_template, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# Model and data preparation constants
MODEL_FILE = "heart_attack_model.h5"
SCALER_FILE = "scaler.pkl"
ENCODER_FILE = "label_encoder.pkl"

# Function to load the trained model and preprocessing objects
def load_model_and_preprocessors():
    model = tf.keras.models.load_model(MODEL_FILE)
    with open(SCALER_FILE, 'rb') as f:
        scaler = pickle.load(f)
    with open(ENCODER_FILE, 'rb') as f:
        label_encoder = pickle.load(f)
    recommendations = {
        "Low": "Maintain a healthy lifestyle with regular exercise and balanced diet.",
        "Moderate": "Consider lifestyle changes and regular check-ups with your doctor.",
        "High": "Please consult a healthcare professional immediately."
    }
    feature_columns = ['Age', 'Gender', 'Heart rate', 'Systolic blood pressure',
                      'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin']
    return model, scaler, label_encoder, recommendations, feature_columns

model, scaler, label_encoder, recommendations, feature_columns = load_model_and_preprocessors()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        age = float(request.form['age'])
        gender = int(request.form['gender'])
        heart_rate = float(request.form['heart_rate'])
        systolic_bp = float(request.form['systolic_bp'])
        diastolic_bp = float(request.form['diastolic_bp'])
        blood_sugar = float(request.form['blood_sugar'])
        ck_mb = float(request.form['ck_mb'])
        troponin = float(request.form['troponin'])

        user_input = np.array([age, gender, heart_rate, systolic_bp,
                               diastolic_bp, blood_sugar, ck_mb, troponin]).reshape(1, -1)
        user_input_scaled = scaler.transform(user_input)

        prediction = model.predict(user_input_scaled, verbose=0)
        risk_index = np.argmax(prediction)
        risk_label = label_encoder.inverse_transform([risk_index])[0]
        recommendation = recommendations[risk_label]
        confidence = float(prediction[0][risk_index] * 100)

        input_summary = {
            "Age": age,
            "Gender": "Male" if gender == 1 else "Female",
            "Heart rate": heart_rate,
            "Systolic blood pressure": systolic_bp,
            "Diastolic blood pressure": diastolic_bp,
            "Blood sugar": blood_sugar,
            "CK-MB": ck_mb,
            "Troponin": troponin
        }

        return jsonify({
            'status': 'success',
            'risk_level': risk_label,
            'confidence': f"{confidence:.2f}%",
            'recommendation': recommendation,
            'input_summary': input_summary
        })

    except Exception as e:
        return jsonify({
            'status': 'error',
            'message': str(e)
        })

if __name__ == '__main__':
    os.makedirs('templates', exist_ok=True)
    with open('templates/index.html', 'w') as f:
        f.write('''
<!DOCTYPE html>
<html lang="en" class="dark">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Heart Risk Predictor</title>
  <script src="https://cdn.tailwindcss.com"></script>
  <script>
    tailwind.config = {
      darkMode: 'class',
      theme: {
        extend: {
          colors: {
            background: '#0e0e10',
            card: '#1c1c1e',
            input: '#2c2c2e',
            text: '#f4f4f5',
            accent: '#4f46e5',
          }
        }
      }
    }
  </script>
</head>
<body class="bg-background text-text min-h-screen px-4 py-8">
  <div class="max-w-3xl mx-auto">
    <h1 class="text-3xl font-bold text-center mb-6">❤️ Heart Attack Risk Prediction</h1>
    <form id="form" class="bg-card p-6 rounded-2xl shadow-xl space-y-4">
      <div class="grid grid-cols-1 md:grid-cols-2 gap-4">
        <div>
          <label class="block mb-1">Age</label>
          <input name="age" type="number" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
        </div>
        <div>
          <label class="block mb-1">Gender</label>
          <select name="gender" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
            <option value="">Select</option>
            <option value="0">Female</option>
            <option value="1">Male</option>
          </select>
        </div>
        <div>
          <label class="block mb-1">Heart Rate</label>
          <input name="heart_rate" type="number" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
        </div>
        <div>
          <label class="block mb-1">Systolic BP</label>
          <input name="systolic_bp" type="number" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
        </div>
        <div>
          <label class="block mb-1">Diastolic BP</label>
          <input name="diastolic_bp" type="number" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
        </div>
        <div>
          <label class="block mb-1">Blood Sugar</label>
          <input name="blood_sugar" type="number" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
        </div>
        <div>
          <label class="block mb-1">CK-MB</label>
          <input name="ck_mb" type="any" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
        </div>
        <div>
          <label class="block mb-1">Troponin</label>
          <input name="troponin" type="any" required class="w-full p-2 bg-input text-text rounded-md focus:ring-2 ring-accent">
        </div>
      </div>
      <button type="submit" class="w-full py-2 bg-accent text-white font-semibold rounded-md hover:bg-indigo-600 transition">Predict</button>
    </form>

    <div id="result" class="hidden mt-6 p-6 rounded-xl shadow-lg space-y-4">
      <h2 class="text-2xl font-bold">Prediction Result</h2>
      <p><strong>Risk Level:</strong> <span id="riskLevel" class="font-semibold"></span></p>
      <p><strong>Confidence:</strong> <span id="confidence"></span></p>
      <p><strong>Recommendation:</strong> <span id="recommendation"></span></p>
      <h3 class="text-xl font-bold mt-4">Your Metrics:</h3>
      <ul id="inputSummary" class="list-disc list-inside space-y-1"></ul>
      <button onclick="location.reload()" class="mt-4 w-full py-2 bg-accent text-white font-semibold rounded-md hover:bg-indigo-600 transition">Predict Again</button>
    </div>
  </div>
  <script>
    document.getElementById('form').addEventListener('submit', async (e) => {
      e.preventDefault();
      const formData = new FormData(e.target);
      const res = await fetch('/predict', { method: 'POST', body: formData });
      const data = await res.json();
      if (data.status === 'success') {
        const resultCard = document.getElementById('result');
        const risk = data.risk_level.toLowerCase();

        // Remove all previous background colors
        resultCard.className = "mt-6 p-6 rounded-xl shadow-lg space-y-4";

        // Apply color based on risk level
        if (risk === "high") {
          resultCard.classList.add("bg-red-600", "text-white");
        } else if (risk === "moderate") {
          resultCard.classList.add("bg-yellow-400", "text-black");
        } else if (risk === "low") {
          resultCard.classList.add("bg-green-500", "text-white");
        }

        document.getElementById('form').classList.add('hidden');
        resultCard.classList.remove('hidden');
        document.getElementById('riskLevel').textContent = data.risk_level;
        document.getElementById('confidence').textContent = data.confidence;
        document.getElementById('recommendation').textContent = data.recommendation;
        const summary = document.getElementById('inputSummary');
        summary.innerHTML = '';
        for (const [k, v] of Object.entries(data.input_summary)) {
          const li = document.createElement('li');
          li.textContent = `${k}: ${v}`;
          summary.appendChild(li);
        }
      } else {
        alert(data.message);
      }
    });
  </script>
</body>
</html>
        ''')
    print("Starting ngrok tunnel...")
    ngrok_tunnel = ngrok.connect(5000)
    public_url = ngrok_tunnel.public_url
    print(f"Public URL: {public_url}")
    app.run()


# PairPLot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load your dataset
df = pd.read_csv("Heart_Attack_Risk_Levels_Dataset.csv")

# Visualize pairwise relationships
sns.pairplot(df, hue="Risk_Level", vars=['Age', 'Heart rate', 'Systolic blood pressure',
                                         'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin'],
             palette='Set2', plot_kws={'alpha':0.6})
plt.suptitle("Pairplot of Heart Attack Risk Features", y=1.02)
plt.show()


# Label Comparisons

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set(style="whitegrid", palette="muted")

feature_columns = ['Age', 'Gender', 'Heart rate', 'Systolic blood pressure',
                  'Diastolic blood pressure', 'Blood sugar', 'CK-MB', 'Troponin']

df = pd.read_csv("Heart_Attack_Risk_Levels_Dataset.csv")
y = df['Risk_Level']  # Assuming 'Risk_Level' is your target column

visual_df = df[feature_columns].copy()
visual_df['Risk_Level'] = y  # original (non-encoded) labels

feature_pairs = [
    ("Age", "CK-MB"),
    ("Troponin", "Heart rate"),
    ("Systolic blood pressure", "Diastolic blood pressure"),
    ("Blood sugar", "CK-MB")
]

plt.figure(figsize=(16, 12))
for i, (feat1, feat2) in enumerate(feature_pairs):
    plt.subplot(2, 2, i + 1)
    sns.scatterplot(data=visual_df, x=feat1, y=feat2, hue="Risk_Level", alpha=0.7, s=60)
    plt.title(f"{feat1} vs {feat2}")
plt.tight_layout()
plt.show()